### Load two different AlexNet Models: 

i)  one model is alexnet (model)

ii) another model is alexnetRT (model_trt)

In [ ]:
from torchvision import models
import torch

import torch
from torch2trt import torch2trt
from torchvision.models.alexnet import alexnet
from torch2trt import TRTModule


model = models.alexnet(pretrained=True)
if torch.cuda.is_available():
    model.to('cuda')


### load alexnetRT m

In [ ]:
#load TRT model
modeltrt = TRTModule()
modeltrt.load_state_dict(torch.load('alexnet_trt.pth'))
device = torch.device('cuda')
model_trt = modeltrt.to(device)


### Get imagenet classification names


In [ ]:
with open('imagenet_classes.txt') as f:
  labels = [line.strip() for line in f.readlines()]

### load image transformation


In [ ]:
from PIL import Image
from torchvision import transforms
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets
from numpy import asarray
from PIL import Image
from IPython.display import Image 

img = widgets.Image(format='jpeg', width=224, height=224)
display(img)

In [ ]:
import io, torch
import PIL
from PIL import Image
import numpy

file = open('turkish.jpg', "rb")
img.value = file.read()
imageStream = io.BytesIO(img.value)
imageFile = Image.open(imageStream)
imgarr = numpy.array(imageFile)

input_image = PIL.Image.fromarray(imgarr)
input_tensor=preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')

In [ ]:
import time
t = time.time()
output = model_trt(input_batch)
index=torch.argmax(output)
print(labels[index])
elapsed = time.time() - t
print(elapsed)

In [ ]:
import time
t = time.time()
output = model(input_batch)
index=torch.argmax(output)
print(labels[index])
elapsed = time.time() - t
print(elapsed)